### Tahap 1: Pemuatan Data dan Inisialisasi

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
# --- Atur Path (Gunakan kode dari notebook 00 yang sudah direvisi) ---
# BASE_DIR, PATH_SPLIT_DATA, PATH_PREDICTIONS, dll.

# --- Muat Data ---
path_split_data = r'C:\MyFolder\Git\TA_SpatioTemporal\Data\split_data' # Path ke folder split_data
train_df = pd.read_parquet(os.path.join(path_split_data, 'train_set.parquet'))
test_df = pd.read_parquet(os.path.join(path_split_data, 'test_set.parquet'))

# --- Definisikan Fitur dan Target ---
TARGET = 'konsumsi_energi'
# Hapus fitur non-numerik atau yang tidak relevan untuk LSTM (seperti 'apakah_akhir_pekan')
FEATURES = [col for col in train_df.columns if col not in ['timestamp', 'meter_id', TARGET]]

print("Data Latih dan Uji berhasil dimuat.")
print("Fitur yang akan digunakan:", FEATURES)


Data Latih dan Uji berhasil dimuat.
Fitur yang akan digunakan: ['is_kelas', 'is_kantor', 'is_penelitian', 'avg_temp_previous_hour', 'jam', 'hari_minggu', 'hari_bulan', 'minggu_tahun', 'bulan', 'tahun', 'apakah_akhir_pekan', 'apakah_jam_kerja', 'konsumsi_lag_1_jam', 'konsumsi_lag_24_jam']


### Tahap 2: Pra-pemrosesan Data

In [4]:
# --- Konfigurasi LSTM ---
N_PAST = 24 # Jumlah jam masa lalu yang digunakan untuk prediksi (contoh)
N_FUTURE = 1 # Memprediksi 1 jam ke depan

X_train, y_train = [], []
X_test, y_test = [], []
scalers = {} # Dictionary untuk menyimpan scaler untuk setiap gedung

# --- Proses Data Latih ---
print("\nMemproses Data Latih...")
for meter_id, group in train_df.groupby('meter_id'):
    # 1. Scaling: Fit dan transform HANYA pada data latih gedung ini
    scaler = MinMaxScaler()
    group_scaled = scaler.fit_transform(group[FEATURES + [TARGET]])
    scalers[meter_id] = scaler # Simpan scaler untuk digunakan pada data uji nanti

    # 2. Buat Sekuens
    for i in range(N_PAST, len(group_scaled) - N_FUTURE + 1):
        X_train.append(group_scaled[i - N_PAST:i, 0:len(FEATURES)])
        y_train.append(group_scaled[i + N_FUTURE - 1:i + N_FUTURE, len(FEATURES)])

# --- Proses Data Uji ---
print("Memproses Data Uji...")
# Simpan indeks asli dari data uji untuk merekonstruksi hasil nanti
test_indices = []
for meter_id, group in test_df.groupby('meter_id'):
    if meter_id in scalers:
        # 1. Scaling: Gunakan scaler dari data latih (HANYA transform)
        scaler = scalers[meter_id]
        group_scaled = scaler.transform(group[FEATURES + [TARGET]])

        # 2. Buat Sekuens
        for i in range(N_PAST, len(group_scaled) - N_FUTURE + 1):
            X_test.append(group_scaled[i - N_PAST:i, 0:len(FEATURES)])
            y_test.append(group_scaled[i + N_FUTURE - 1:i + N_FUTURE, len(FEATURES)])
            # Simpan indeks baris asli dari DataFrame test_df
            test_indices.append(group.index[i + N_FUTURE - 1])

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

print(f"\nBentuk data latih (X, y): {X_train.shape}, {y_train.shape}")
print(f"Bentuk data uji (X, y): {X_test.shape}, {y_test.shape}")



Memproses Data Latih...
Memproses Data Uji...

Bentuk data latih (X, y): (282555, 24, 14), (282555, 1)
Bentuk data uji (X, y): (47541, 24, 14), (47541, 1)


### Tahap 3: Pelatihan Model LSTM

In [5]:
# --- Bangun Arsitektur Model ---
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dense(y_train.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

# --- Latih Model ---
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)


c:\Users\Danu Hakim\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 24, 64)         │        20,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,673 (127.63 KB)

 Trainable params: 32,673 (127.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
7947/7947 ━━━━━━━━━━━━━━━━━━━━ 93s 11ms/step - loss: 0.0068 - val_loss: 0.0042
Epoch 2/10
7947/7947 ━━━━━━━━━━━━━━━━━━━━ 92s 12ms/step - loss: 0.0036 - val_loss: 0.0040
Epoch 3/10
7947/7947 ━━━━━━━━━━━━━━━━━━━━ 97s 12ms/step - loss: 0.0034 - val_loss: 0.0039
Epoch 4/10
7947/7947 ━━━━━━━━━━━━━━━━━━━━ 111s 14ms/step - loss: 0.0033 - val_loss: 0.0042
Epoch 5/10
7947/7947 ━━━━━━━━━━━━━━━━━━━━ 110s 14ms/step - loss: 0.0032 - val_loss: 0.0045
Epoch 6/10
7947/7947 ━━━━━━━━━━━━━━━━━━━━ 111s 14ms/step - loss: 0.0032 - val_loss: 0.0045
Epoch 7/10
7947/7947 ━━━━━━━━━━━━━━━━━━━━ 112s 14ms/step - loss: 0.0031 - val_loss: 0.0040
Epoch 8/10
7947/7947 ━━━━━━━━━━━━━━━━━━━━ 114s 14ms/step - loss: 0.0031 - val_loss: 0.0038
Epoch 9/10
7947/7947 ━━━━━━━━━━━━━━━━━━━━ 113s 14ms/step - loss: 0.0030 - val_loss: 0.0040
Epoch 10/10
7947/7947 ━━━━━━━━━━━━━━━━━━━━ 113s 14ms/step - loss: 0.0030 - val_loss: 0.0044


### Tahap 4: Prediksi dan Rekonstruksi Hasil (Paling Kritis)

In [7]:
# --- Lakukan Prediksi ---
predictions_scaled = model.predict(X_test)

# --- Buat DataFrame Hasil yang Solid ---
# 1. Gunakan indeks yang sudah kita simpan untuk memastikan kesejajaran
df_hasil = test_df.loc[test_indices].copy()

# 2. Siapkan array untuk hasil yang sudah di-inverse-transform
y_pred_inversed = np.array([])
y_test_inversed = np.array([])

# 3. Lakukan inverse transform per gedung
for meter_id, group in df_hasil.groupby('meter_id'):
    if meter_id in scalers:
        # Ambil indeks baris untuk gedung ini
        group_indices = group.index
        
        # Cari posisi prediksi yang sesuai dengan indeks ini
        posisi = [test_indices.index(i) for i in group_indices]

        # Ambil prediksi dan nilai aktual yang relevan
        preds_scaled_group = predictions_scaled[posisi]
        test_scaled_group = y_test[posisi]
        
        # Buat dummy array untuk inverse transform
        dummy_pred = np.zeros((len(preds_scaled_group), len(FEATURES) + 1))
        dummy_pred[:, -1] = preds_scaled_group.ravel()
        
        dummy_test = np.zeros((len(test_scaled_group), len(FEATURES) + 1))
        dummy_test[:, -1] = test_scaled_group.ravel()

        # Inverse transform
        inversed_preds = scalers[meter_id].inverse_transform(dummy_pred)[:, -1]
        inversed_tests = scalers[meter_id].inverse_transform(dummy_test)[:, -1]
        
        # Tambahkan ke array hasil total
        y_pred_inversed = np.append(y_pred_inversed, inversed_preds)
        y_test_inversed = np.append(y_test_inversed, inversed_tests)

# 4. Tambahkan kolom hasil ke DataFrame
df_hasil['prediksi_lstm'] = y_pred_inversed
df_hasil.rename(columns={TARGET: 'target_aktual'}, inplace=True)

# --- Evaluasi Akhir (Sekarang Seharusnya Konsisten) ---
mae = mean_absolute_error(df_hasil['target_aktual'], df_hasil['prediksi_lstm'])
r2 = r2_score(df_hasil['target_aktual'], df_hasil['prediksi_lstm'])
print(f"\nEvaluasi Final yang Konsisten:")
print(f"MAE: {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

# --- Simpan Hasil yang Sudah Benar ---
path_prediksi = r'C:\MyFolder\Git\TA_SpatioTemporal\Data\predictions' # Path ke folder predictions
df_hasil[['timestamp', 'meter_id', 'target_aktual', 'prediksi_lstm']].to_parquet(
    os.path.join(path_prediksi, 'lstm_results_revised.parquet'), index=False
)
print("\nDataFrame hasil LSTM yang sudah sejajar berhasil disimpan.")


1486/1486 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step

Evaluasi Final yang Konsisten:
MAE: 2.4840
R-squared (R²): 0.9579

DataFrame hasil LSTM yang sudah sejajar berhasil disimpan.
